In [1]:
import xgboost as xgb
import pandas as pd
import datetime

import pickle

In [2]:
df_all = pd.read_csv("./datasets/stock_20200821.csv")

# # 保留指数的pct_chg 和 成交量特征
# for ci in df_all.columns:
#     if ci.endswith("_sh") and (not ci.startswith("pct_chg") and not ci.startswith("vol")):
#         del df_all[ci]

# for ci in df_all.columns:
#     if ci.endswith("_sh"):
#         del df_all[ci]

df_all = df_all[['trade_date','pricevolma5', 'pct_chg', 'code']]
print(df_all.columns)
print(df_all.shape)

Index(['trade_date', 'pricevolma5', 'pct_chg', 'code'], dtype='object')
(883844, 4)


In [3]:
df_all["pct_chg"] = df_all.groupby("code")["pct_chg"].shift(-2)

df_all.dropna(inplace=True)

df_all["trade_date"] = df_all["trade_date"].apply(lambda x: str(x))
df_all["trade_date"] = pd.to_datetime(df_all["trade_date"])

return_list = []
datelist = []
stock_list = []
score_train_list = []
score_valid_list = []

date_list = list(set(df_all["trade_date"]))
date_list.sort()
date_dict = {di: idx for idx, di in enumerate(date_list)}

In [4]:
for ci in df_all.columns:
    print(ci, df_all[ci].dtype)

trade_date datetime64[ns]
pricevolma5 float64
pct_chg float64
code object


# debug

In [5]:
import numpy as np


today_data = df_all[df_all["trade_date"] == datetime.datetime(2016, 6, 6)]
today_date = today_data.iloc[0]["trade_date"]

test_end_date = date_list[date_dict[today_date] - 1]
test_start_date = date_list[date_dict[today_date] - 21]
train_end_date = date_list[date_dict[today_date] - 22]
train_start_date = date_list[date_dict[today_date] - 82]
df_sh_train = df_all[
    (df_all["trade_date"] >= train_start_date) & (
        df_all["trade_date"] <= train_end_date)]
df_sh_train_X = df_sh_train.values[:, 1:-2].astype(np.float)
df_sh_train_y = df_sh_train.values[:, -2].astype(np.float)

df_sh_test = df_all[
    (df_all["trade_date"] >= test_start_date) & (
        df_all["trade_date"] < test_end_date)]
df_sh_test_X = df_sh_test.values[:, 1:-2].astype(np.float)
df_sh_test_y = df_sh_test.values[:, -2].astype(np.float)

In [11]:
df_sh_train_X

array([[10.19565878],
       [ 3.10678578],
       [24.23385189],
       ...,
       [17.99122906],
       [16.54912439],
       [13.07780553]])

In [12]:
df_sh_train_X.shape

(17085, 1)

In [22]:
delta_days = (df_sh_train["trade_date"].max() -
              df_sh_train["trade_date"].min()).days

delta_days = (df_sh_test["trade_date"].max() -
              df_sh_test["trade_date"].min()).days

myxgb = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=15)
eval_set = [(df_sh_test_X, df_sh_test_y)]
myxgb.fit(df_sh_train_X, df_sh_train_y, eval_metric="rmse", eval_set=eval_set, verbose=False,
          early_stopping_rounds=100)

train_score = myxgb.score(df_sh_train_X, df_sh_train_y)
valid_score = myxgb.score(df_sh_test_X, df_sh_test_y)


X = today_data.values[:, 1:-2].astype(np.float)
y_pred = myxgb.predict(X)

df = pd.DataFrame({"stock": today_data["code"], "y_pred": y_pred})
print(df)

            stock    y_pred
588073  600000.SH  0.001410
588074  600008.SH  0.003166
588075  600009.SH  0.002886
588076  600010.SH  0.003338
588077  600011.SH  0.005615
588078  600015.SH  0.003552
588079  600016.SH  0.002576
588080  600018.SH  0.002841
588081  600019.SH  0.002841
588082  600023.SH  0.002841
588083  600027.SH  0.002841
588084  600028.SH  0.003166
588085  600029.SH  0.005886
588086  600030.SH  0.002265
588087  600031.SH  0.003644
588088  600036.SH  0.001465
588089  600038.SH -0.010706
588090  600048.SH  0.003552
588091  600050.SH  0.003112
588092  600058.SH  0.001000
588093  600060.SH  0.002265
588094  600066.SH  0.001292
588095  600068.SH  0.001456
588096  600079.SH -0.000884
588097  600085.SH  0.001523
588098  600089.SH  0.003552
588099  600100.SH  0.001140
588100  600104.SH  0.004239
588101  600108.SH  0.004052
588102  600109.SH  0.000879
...           ...       ...
588328  002375.SZ  0.002765
588329  002385.SZ  0.003663
588330  002399.SZ  0.002288
588331  002400.SZ  0

In [25]:
delta_days = (df_sh_train["trade_date"].max() -
              df_sh_train["trade_date"].min()).days
sample_weight = (df_sh_train["trade_date"] - df_sh_train["trade_date"].min()
                 ).apply(lambda x: (x.days + delta_days))
sample_weight = np.array(sample_weight)

delta_days = (df_sh_test["trade_date"].max() -
              df_sh_test["trade_date"].min()).days
sample_weight_eval = (
    df_sh_test["trade_date"] - df_sh_test["trade_date"].min()).apply(lambda x: (x.days + delta_days))
sample_weight_eval = np.array(sample_weight_eval)

myxgb = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.01, max_depth=15)
eval_set = [(df_sh_test_X, df_sh_test_y)]
myxgb.fit(df_sh_train_X, df_sh_train_y, eval_metric="rmse", eval_set=eval_set, verbose=False,
          early_stopping_rounds=100, sample_weight=sample_weight,
          sample_weight_eval_set=[sample_weight_eval])

train_score = myxgb.score(df_sh_train_X, df_sh_train_y)
valid_score = myxgb.score(df_sh_test_X, df_sh_test_y)


X = today_data.values[:, 1:-2].astype(np.float)
y_pred = myxgb.predict(X)

df = pd.DataFrame({"stock": today_data["code"], "y_pred": y_pred})
print(df)

            stock    y_pred
588073  600000.SH  0.003527
588074  600008.SH  0.003512
588075  600009.SH  0.005907
588076  600010.SH  0.003717
588077  600011.SH  0.005446
588078  600015.SH  0.005404
588079  600016.SH  0.004277
588080  600018.SH  0.003928
588081  600019.SH  0.003814
588082  600023.SH -0.000084
588083  600027.SH  0.006754
588084  600028.SH  0.006200
588085  600029.SH  0.006648
588086  600030.SH  0.004989
588087  600031.SH  0.005402
588088  600036.SH  0.001610
588089  600038.SH -0.002512
588090  600048.SH  0.005407
588091  600050.SH  0.005406
588092  600058.SH  0.003816
588093  600060.SH  0.004989
588094  600066.SH  0.003567
588095  600068.SH -0.000095
588096  600079.SH  0.001140
588097  600085.SH  0.004179
588098  600089.SH  0.005407
588099  600100.SH  0.004249
588100  600104.SH  0.006402
588101  600108.SH  0.008651
588102  600109.SH  0.006140
...           ...       ...
588328  002375.SZ  0.005354
588329  002385.SZ  0.005461
588330  002399.SZ  0.004251
588331  002400.SZ  0

In [24]:
sample_weight.sum()

1.0

In [52]:
X = X.astype(float)
X[1]

array([ 1.00518774e+00,  1.00842657e+00,  1.42274000e+01,  1.22998148e-01,
        3.42067485e-02,  1.44845047e+01,  1.42653301e+01,  1.39224079e+01,
        1.36317806e+01,  1.41445335e+01,  1.45412774e+01,  1.37477896e+01,
        6.64007732e-01,  5.97213305e-01,  3.64079666e-01, -2.58715930e-03,
        2.73170521e-02,  3.42067485e-02,  5.61684610e-02,  1.89820414e-01,
        1.58164973e-02, -3.99886409e-01,  1.31425551e-01,  2.86881052e-01,
        3.36676906e-01,  4.51528051e-01, -4.95670823e-01,  5.79454158e-01,
        4.67817508e-01,  3.65118448e-01,  1.10405971e+00,  2.36760024e+00,
        1.75091379e+00,  1.65388525e+00,  3.37706698e+00])

In [36]:
myxgb.predict(X[2].reshape(1, -1))

array([0.00454798], dtype=float32)

In [39]:
myxgb.feature_importances_

F:\mypython\Anaconda3\lib\site-packages\xgboost\sklearn.py:695: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

# 训练

In [6]:
import numpy as np

for year in range(2015, 2021):
    for month in range(1, 13):
        for i in range(1, 32):
            try:
                today_data = df_all[df_all["trade_date"]
                                    == datetime.datetime(year, month, i)]
            except:
                continue

            if len(today_data) == 0:
                continue

            today_date = today_data.iloc[0]["trade_date"]

            test_end_date = date_list[date_dict[today_date] - 1]
            test_start_date = date_list[date_dict[today_date] - 21]
            train_end_date = date_list[date_dict[today_date] - 22]
            train_start_date = date_list[date_dict[today_date] - 82]
            df_sh_train = df_all[
                (df_all["trade_date"] >= train_start_date) & (
                    df_all["trade_date"] <= train_end_date)]
            df_sh_train_X = df_sh_train.values[:, 1:-2]
            df_sh_train_y = df_sh_train.values[:, -2]

            df_sh_test = df_all[
                (df_all["trade_date"] >= test_start_date) & (
                    df_all["trade_date"] < test_end_date)]
            df_sh_test_X = df_sh_test.values[:, 1:-2]
            df_sh_test_y = df_sh_test.values[:, -2]

            delta_days = (df_sh_train["trade_date"].max(
            ) - df_sh_train["trade_date"].min()).days
            sample_weight = (
                df_sh_train["trade_date"] - df_sh_train["trade_date"].min()).apply(lambda x: (x.days + delta_days))
            sample_weight = np.array(sample_weight / sample_weight.sum())

            delta_days = (df_sh_test["trade_date"].max(
            ) - df_sh_test["trade_date"].min()).days
            sample_weight_eval = (
                df_sh_test["trade_date"] - df_sh_test["trade_date"].min()).apply(lambda x: (x.days + delta_days))
            sample_weight_eval = np.array(
                sample_weight_eval / sample_weight_eval.sum())

            myxgb = xgb.XGBRegressor(
                n_estimators=1000, learning_rate=0.01, max_depth=15)
            eval_set = [(df_sh_test_X, df_sh_test_y)]
            myxgb.fit(df_sh_train_X, df_sh_train_y, eval_metric="rmse", eval_set=eval_set, verbose=False,
                      early_stopping_rounds=100, sample_weight=sample_weight,
                      sample_weight_eval_set=[sample_weight_eval])

            train_score = myxgb.score(df_sh_train_X, df_sh_train_y)
            valid_score = myxgb.score(df_sh_test_X, df_sh_test_y)

            X = today_data.values[:, 1:-2]
            y_pred = myxgb.predict(X)

            df = pd.DataFrame({"stock": today_data["code"], "y_pred": y_pred})
            print(df)
            df = df[df["y_pred"] > 0.05]
            df.sort_values(by="y_pred", ascending=False, inplace=True)
            df.reset_index(drop=True, inplace=True)

            tmp = 0
            for stocki in df["stock"]:
                return_ = today_data[today_data["code"]
                                     == stocki].values[0, -2]
                tmp += return_

            if len(df) == 0:
                return_list.append(0)
            else:
                return_list.append(tmp / len(df))

            datelist.append(today_data.iloc[0]["trade_date"])
            stock_list.append(df["stock"].tolist())
            score_train_list.append(train_score)
            score_valid_list.append(valid_score)

            print("train score:", train_score)
            print("valid score:", valid_score)
            print(today_data.iloc[0]["trade_date"])
            print(df["stock"].tolist())
            print("return:", return_list[-1])
            print("----------------------------------------")

            stock   y_pred
252692  600705.SH  0.00887
252693  600837.SH  0.00887
252694  600703.SH  0.00887
252695  600066.SH  0.00887
252696  600606.SH  0.00887
252697  600655.SH  0.00887
252698  600660.SH  0.00887
252699  600741.SH  0.00887
252700  600795.SH  0.00887
252701  600809.SH  0.00887
252702  600867.SH  0.00887
252703  600886.SH  0.00887
252704  600887.SH  0.00887
252705  600893.SH  0.00887
252706  600009.SH  0.00887
252707  600109.SH  0.00887
252708  600111.SH  0.00887
252709  600115.SH  0.00887
252710  600196.SH  0.00887
252711  600816.SH  0.00887
252712  600153.SH  0.00887
252713  600674.SH  0.00887
252714  600690.SH  0.00887
252715  600760.SH  0.00887
252716  600848.SH  0.00887
252717  600000.SH  0.00887
252718  600104.SH  0.00887
252719  600118.SH  0.00887
252720  600170.SH  0.00887
252721  600177.SH  0.00887
...           ...      ...
252807  601727.SH  0.00887
252808  601668.SH  0.00887
252809  601788.SH  0.00887
252810  601618.SH  0.00887
252811  601888.SH  0.00887
2

            stock    y_pred
253423  600705.SH  0.009682
253424  600837.SH  0.009682
253425  600703.SH  0.009682
253426  600066.SH  0.009682
253427  600606.SH  0.009682
253428  600655.SH  0.009682
253429  600660.SH  0.009682
253430  600741.SH  0.009682
253431  600795.SH  0.009682
253432  600809.SH  0.009682
253433  600867.SH  0.009682
253434  600886.SH  0.009682
253435  600887.SH  0.009682
253436  600893.SH  0.009682
253437  600009.SH  0.009682
253438  600109.SH  0.009682
253439  600111.SH  0.009682
253440  600115.SH  0.009682
253441  600196.SH  0.009682
253442  600816.SH  0.009682
253443  600153.SH  0.009682
253444  600674.SH  0.009682
253445  600690.SH  0.009682
253446  600760.SH  0.009682
253447  600848.SH  0.009682
253448  600000.SH  0.009682
253449  600061.SH  0.009682
253450  600104.SH  0.009682
253451  600118.SH  0.009682
253452  600170.SH  0.009682
...           ...       ...
253539  601668.SH  0.009682
253540  601788.SH  0.009682
253541  601618.SH  0.009682
253542  601888.SH  0

            stock    y_pred
254157  600705.SH  0.010379
254158  600837.SH  0.010379
254159  600703.SH  0.010379
254160  600066.SH  0.010379
254161  600606.SH  0.010379
254162  600655.SH  0.010379
254163  600660.SH  0.010379
254164  600741.SH  0.010379
254165  600795.SH  0.010379
254166  600809.SH  0.010379
254167  600867.SH  0.010379
254168  600886.SH  0.010379
254169  600887.SH  0.010379
254170  600893.SH  0.010379
254171  600009.SH  0.010379
254172  600109.SH  0.010379
254173  600111.SH  0.010379
254174  600115.SH  0.010379
254175  600196.SH  0.010379
254176  600816.SH  0.010379
254177  600153.SH  0.010379
254178  600674.SH  0.010379
254179  600690.SH  0.010379
254180  600760.SH  0.010379
254181  600848.SH  0.010379
254182  600000.SH  0.010379
254183  600061.SH  0.010379
254184  600104.SH  0.010379
254185  600118.SH  0.010379
254186  600170.SH  0.010379
...           ...       ...
254275  601727.SH  0.010379
254276  601668.SH  0.010379
254277  601788.SH  0.010379
254278  601618.SH  0

            stock    y_pred
254897  600705.SH  0.009462
254898  600837.SH  0.009462
254899  600703.SH  0.009462
254900  600066.SH  0.009462
254901  600606.SH  0.009462
254902  600655.SH  0.009462
254903  600660.SH  0.009462
254904  600741.SH  0.009462
254905  600795.SH  0.009462
254906  600809.SH  0.009462
254907  600867.SH  0.009462
254908  600886.SH  0.009462
254909  600887.SH  0.009462
254910  600893.SH  0.009462
254911  600009.SH  0.009462
254912  600109.SH  0.009462
254913  600111.SH  0.009462
254914  600115.SH  0.009462
254915  600196.SH  0.009462
254916  600816.SH  0.009462
254917  600153.SH  0.009462
254918  600674.SH  0.009462
254919  600690.SH  0.009462
254920  600760.SH  0.009462
254921  600848.SH  0.009462
254922  600000.SH  0.009462
254923  600061.SH  0.009462
254924  600104.SH  0.009462
254925  600118.SH  0.009462
254926  600170.SH  0.009462
...           ...       ...
255015  601727.SH  0.009462
255016  601668.SH  0.009462
255017  601788.SH  0.009462
255018  601618.SH  0

            stock    y_pred
255629  600705.SH  0.010336
255630  600837.SH  0.010336
255631  600703.SH  0.010336
255632  600066.SH  0.010336
255633  600606.SH  0.010336
255634  600655.SH  0.010336
255635  600660.SH  0.010336
255636  600741.SH  0.010336
255637  600795.SH  0.010336
255638  600809.SH  0.010336
255639  600867.SH  0.010336
255640  600886.SH  0.010336
255641  600887.SH  0.010336
255642  600893.SH  0.010336
255643  600009.SH  0.010336
255644  600109.SH  0.010336
255645  600111.SH  0.010336
255646  600115.SH  0.010336
255647  600196.SH  0.010336
255648  600816.SH  0.010336
255649  600153.SH  0.010336
255650  600674.SH  0.010336
255651  600690.SH  0.010336
255652  600760.SH  0.010336
255653  600848.SH  0.010336
255654  600000.SH  0.010336
255655  600061.SH  0.010336
255656  600104.SH  0.010336
255657  600118.SH  0.010336
255658  600170.SH  0.010336
...           ...       ...
255744  601766.SH  0.010336
255745  601727.SH  0.010336
255746  601668.SH  0.010336
255747  601618.SH  0

...

            stock    y_pred
274971  600705.SH  0.002533
274972  600837.SH  0.002533
274973  600703.SH  0.002533
274974  600066.SH  0.002533
274975  600606.SH  0.002533
274976  600655.SH  0.002533
274977  600660.SH  0.002533
274978  600741.SH  0.002533
274979  600795.SH  0.002533
274980  600809.SH  0.002533
274981  600867.SH  0.002533
274982  600887.SH  0.002533
274983  600893.SH  0.002533
274984  600009.SH  0.002533
274985  600109.SH  0.002533
274986  600111.SH  0.002533
274987  600115.SH  0.002533
274988  600196.SH  0.002533
274989  600816.SH  0.002533
274990  600674.SH  0.002533
274991  600690.SH  0.002533
274992  600760.SH  0.002533
274993  600848.SH  0.002533
274994  600000.SH  0.002533
274995  600061.SH  0.002533
274996  600104.SH  0.002533
274997  600118.SH  0.002533
274998  600170.SH  0.002533
274999  600177.SH  0.002533
275000  600188.SH  0.002533
...           ...       ...
275083  601727.SH  0.002533
275084  601668.SH  0.002533
275085  601788.SH  0.002533
275086  601888.SH  0

            stock    y_pred
275682  600705.SH  0.001373
275683  600837.SH  0.001373
275684  600703.SH  0.001373
275685  600066.SH  0.001373
275686  600606.SH  0.001373
275687  600655.SH  0.001373
275688  600660.SH  0.001373
275689  600741.SH  0.001373
275690  600795.SH  0.001373
275691  600809.SH  0.001373
275692  600867.SH  0.001373
275693  600887.SH  0.001373
275694  600893.SH  0.001373
275695  600009.SH  0.001373
275696  600109.SH  0.001373
275697  600111.SH  0.001373
275698  600115.SH  0.001373
275699  600196.SH  0.001373
275700  600816.SH  0.001373
275701  600674.SH  0.001373
275702  600690.SH  0.001373
275703  600760.SH  0.001373
275704  600848.SH  0.001373
275705  600000.SH  0.001373
275706  600061.SH  0.001373
275707  600104.SH  0.001373
275708  600118.SH  0.001373
275709  600170.SH  0.001373
275710  600177.SH  0.001373
275711  600188.SH  0.001373
...           ...       ...
275794  601727.SH  0.001373
275795  601668.SH  0.001373
275796  601788.SH  0.001373
275797  601888.SH  0

            stock    y_pred
276392  600705.SH  0.000982
276393  600837.SH  0.000982
276394  600703.SH  0.000982
276395  600066.SH  0.000982
276396  600606.SH  0.000982
276397  600655.SH  0.000982
276398  600660.SH  0.000982
276399  600741.SH  0.000982
276400  600795.SH  0.000982
276401  600809.SH  0.000982
276402  600867.SH  0.000982
276403  600887.SH  0.000982
276404  600893.SH  0.000982
276405  600009.SH  0.000982
276406  600109.SH  0.000982
276407  600111.SH  0.000982
276408  600115.SH  0.000982
276409  600196.SH  0.000982
276410  600816.SH  0.000982
276411  600674.SH  0.000982
276412  600690.SH  0.000982
276413  600760.SH  0.000982
276414  600848.SH  0.000982
276415  600000.SH  0.000982
276416  600061.SH  0.000982
276417  600104.SH  0.000982
276418  600118.SH  0.000982
276419  600170.SH  0.000982
276420  600177.SH  0.000982
276421  600188.SH  0.000982
...           ...       ...
276504  601727.SH  0.000982
276505  601668.SH  0.000982
276506  601788.SH  0.000982
276507  601888.SH  0

            stock    y_pred
277102  600705.SH  0.002819
277103  600837.SH  0.002819
277104  600703.SH  0.002819
277105  600066.SH  0.002819
277106  600606.SH  0.002819
277107  600655.SH  0.002819
277108  600660.SH  0.002819
277109  600741.SH  0.002819
277110  600795.SH  0.002819
277111  600809.SH  0.002819
277112  600867.SH  0.002819
277113  600887.SH  0.002819
277114  600893.SH  0.002819
277115  600009.SH  0.002819
277116  600109.SH  0.002819
277117  600111.SH  0.002819
277118  600115.SH  0.002819
277119  600196.SH  0.002819
277120  600816.SH  0.002819
277121  600674.SH  0.002819
277122  600690.SH  0.002819
277123  600760.SH  0.002819
277124  600848.SH  0.002819
277125  600000.SH  0.002819
277126  600061.SH  0.002819
277127  600104.SH  0.002819
277128  600118.SH  0.002819
277129  600170.SH  0.002819
277130  600177.SH  0.002819
277131  600188.SH  0.002819
...           ...       ...
277211  601766.SH  0.002819
277212  601727.SH  0.002819
277213  601668.SH  0.002819
277214  601788.SH  0

KeyboardInterrupt: 

In [7]:
df_sh_test_X.shape

(5668, 35)

In [ ]:
delta_days = (df_sh_train["trade_date"].max() -
              df_sh_train["trade_date"].min()).days
sample_weight = (df_sh_train["trade_date"] - df_sh_train["trade_date"].min()
                 ).apply(lambda x: (x.days + delta_days))
sample_weight /= sample_weight.sum()
print(sample_weight)

In [ ]:
np.isnan(df_sh_test_X)

In [ ]:
np.std(df_sh_train_X, axis=0)